# TF-IDF

출쳐별(slack, nsmc, petition)로 TF-IDF 점수가 높은 상위 20개의 단어들을 찾아보고, TF-IDF 점수가 높은 상위 10개의 단어를 그래프를 통해 보여줍니다. dev.csv / train.csv를 각각 따로 점수를 매김으로써, 둘의 TF-IDF 차이를 서로 비교해봅니다. 

이 작업을 통해 TF-IDF가 높은 단어들이 무엇인지 파악하고, 이를 기반으로 소스 별로 서로 다른 전처리를 하는 편이 낫다는 생각을 하게 되었습니다. 뿐만 아니라 동의어를 통해 데이터 증강을 시도할 때 TF-IDF가 높은 단어를 위주로 진행해보면 효과적인 성능 향상을 가져올 것이라는 생각을 했습니다.

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from konlpy.tag import Okt  # 한국어 형태소 분석기
import re

# 데이터 불러오기
data = pd.read_csv('../../data/dev.csv')
# data = pd.read_csv('../../data/train.csv') - 학습 전용

# 각 소스별로 데이터를 나누기
sources = ['slack', 'nsmc', 'petition'] 

# slcak, nsmc, petition 데이터를 나누기
source_data = {source: data[data['source'].str.contains(source)] for source in sources}

# TF-IDF 벡터라이저 정의
vectorizer = TfidfVectorizer(max_features=500)

# 소스별로 TF-IDF 계산하고 상위 단어 추출하는 함수
def get_top_keywords(tfidf_matrix, terms, n=10): 
		# 차원 변환 및 내림차순 정렬
    sorted_indices = tfidf_matrix.sum(axis=0).A1.argsort()[::-1] 

		# 상위 10개 항목만 추출
    return [terms[i] for i in sorted_indices[:n]]

# 각 소스별로 상위 키워드를 분석하고 시각화
for source, df in source_data.items():
    texts = df['sentence_1'] + " " + df['sentence_2']  # 두 문장을 합치기
    tfidf_matrix = vectorizer.fit_transform(texts)
    terms = vectorizer.get_feature_names_out()
    top_keywords = get_top_keywords(tfidf_matrix, terms)
    
    # 상위 키워드 시각화
    plt.figure(figsize=(10, 6))
    plt.barh(top_keywords, tfidf_matrix.sum(axis=0).A1[tfidf_matrix.sum(axis=0).A1.argsort()[::-1][:10]])
    plt.title(f'{source}의 상위 TF-IDF 단어')
    plt.xlabel('TF-IDF 값')
    plt.ylabel('단어')
    plt.gca().invert_yaxis()  # 상위 단어가 위로 오게 하기
    plt.show()

## 결과 (slack) 

### train.csv

![tf_idf_slack_train](images/tf_idf_slack_train.png)

### dev.csv

![tf_idf_slack_dev](images/tf_idf_slack_dev.png)

## 결과 (nsmc) 

### train.csv

![tf_idf_nsmc_train](images/tf_idf_nsmc_train.png)

### dev.csv

![tf_idf_nsmc_dev](images/tf_idf_nsmc_dev.png)

## 결과 (petition) 

### train.csv

![tf_idf_petition_train](images/tf_idf_petition_train.png)

### dev.csv

![tf_idf_petition_dev](images/tf_idf_petition_dev.png)